Getting the Relevant Assessment Matches:

In [89]:
import requests
import time
import json

In [90]:
TOGETHER_API_KEY = "2b0cb3216fced71ce25a4f28fe37dbd82e6383dd24b20a7e4ab5047b0b485db7"  # Replace with your key
MODEL = "mistralai/Mixtral-8x7B-Instruct-v0.1"

In [91]:
def evaluate_assessment_relevance(assessment_name, job_description):
    prompt = f"""Given the following job description:
\"\"\"
{job_description}
\"\"\"

Evaluate the relevance of the assessment titled: "{assessment_name}"

Respond ONLY in the following JSON format:
{{
  "relevance_score": <integer from 1 to 10>,
  "explanation": "<short explanation>"
}}"""

    try:
        response = requests.post(
            "https://api.together.xyz/v1/completions",
            headers={
                "Authorization": f"Bearer {TOGETHER_API_KEY}",
                "Content-Type": "application/json"
            },
            json={
                "model": MODEL,
                "prompt": prompt,
                "max_tokens": 256,
                "temperature": 0.3,
                "stop": ["```"]
            }
        )

        response.raise_for_status()
        result = response.json()['choices'][0]['text'].strip()

        # Try parsing the JSON
        try:
            data = json.loads(result)
            return data["relevance_score"], data["explanation"]
        except json.JSONDecodeError:
            print(f"❌ Invalid JSON for: {assessment_name}")
            print(f"🔎 Raw response text:\n{result}")
            return None, result

    except requests.exceptions.RequestException as e:
        print(f"❌ Error with {assessment_name}: {e}")
        return None, str(e)

In [92]:
def run_batch_evaluation(assessment_names, job_description):
    results = []

    for assessment in assessment_names:
        print(f"🔍 Evaluating: {assessment}")
        score, explanation = evaluate_assessment_relevance(assessment, job_description)

        results.append({
            "Assessment Name": assessment,
            "Relevance Score": score,
            "Explanation": explanation
        })

        time.sleep(2)  # optional pause to avoid rate-limiting

    return results

In [94]:
job_description = """
We are seeking a Senior .NET Developer with strong experience in ASP.NET Core, Angular, Azure DevOps, and CI/CD pipelines. The candidate should also
be well-versed in C#, RESTful APIs, and cloud-native practices.
"""

In [95]:
results = run_batch_evaluation(assessment_names, job_description)

🔍 Evaluating: Global Skills Development Report
🔍 Evaluating: .NET Framework 4.5
🔍 Evaluating: .NET MVC (New)
🔍 Evaluating: .NET MVVM (New)
🔍 Evaluating: .NET WCF (New)
🔍 Evaluating: .NET WPF (New)
🔍 Evaluating: .NET XAML (New)
🔍 Evaluating: Accounts Payable (New)
🔍 Evaluating: Accounts Payable Simulation (New)
🔍 Evaluating: Accounts Receivable (New)
🔍 Evaluating: Accounts Receivable Simulation (New)
🔍 Evaluating: ADO.NET (New)
🔍 Evaluating: Adobe Experience Manager (New)
🔍 Evaluating: Adobe Photoshop CC
🔍 Evaluating: Aeronautical Engineering (New)
🔍 Evaluating: Aerospace Engineering (New)
🔍 Evaluating: Agile Software Development
🔍 Evaluating: Agile Testing (New)
🔍 Evaluating: AI Skills
🔍 Evaluating: Amazon Web Services (AWS) Development (New)
🔍 Evaluating: Android Development (New)
🔍 Evaluating: Angular 6 (New)
🔍 Evaluating: AngularJS (New)
🔍 Evaluating: Apache Hadoop (New)
🔍 Evaluating: Apache Hadoop Extensions (New)
🔍 Evaluating: Apache HBase (New)
🔍 Evaluating: Apache Hive (New)
🔍 E

In [96]:
# Print nicely
import pandas as pd
df = pd.DataFrame(results)
print(df.sort_values(by="Relevance Score", ascending=False).head(10))

                                   Assessment Name  Relevance Score  \
325                    Smart Interview Live Coding                8   
340                     RESTful Web Services (New)                8   
21                                 Angular 6 (New)                8   
30                          ASP .NET with C# (New)                8   
82       Digital Readiness Development Report - IC                8   
273  Verify - Technical Checking - Next Generation                8   
326                      Smart Interview On Demand                7   
324                           Smart Interview Live                7   
321                      SHL Verify Interactive G+                7   
360                       Reading Comprehension v2                7   

                                           Explanation  
325  The assessment titled 'Smart Interview Live Co...  
340  The assessment title 'RESTful Web Services (Ne...  
21   Angular 6 is a relevant skill for a Senior